# Objectif de charger les données 

In [1]:
import pandas as pd
import os
import json
import glob
import sqlite3, sqlalchemy

In [2]:
#lire le fichier json des paramètres 
with open('../parameters.json') as json_data:
    params = json.load(json_data)
    
DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']
PRICE_FILE = params['FILES']['PRICE_FILE']
STATION_FILE = params['FILES']['STATION_FILE']
SERVICE_FILE = params['FILES']['SERVICE_FILE']
#ici on en profite pour faire tt de suite le path os join
GAS_DB = os.path.join(DATA_DIR, params['DATABASES']['GAS_DB'])

In [13]:
#ce code si on avait pas utilsier le fichier param json

#constante, on met souvent en majusucles. 
#utiliser nom anglais et simple pour sauvegarder les path qu'on va utiliser
#DATA_DIR = 'C:/Users/t15021596/Hands-on-Project---CT/data'
#ou utiliser l emode relatif, ici poru au autre dossier tmp
#TMP_DIR = '../../tmp'

In [6]:
gas_files = glob.glob(os.path.join(DATA_DIR, "Prix*.zip"))
stations_file = 'stations2016.zip'
services_file = 'services2016.zip'

In [15]:
gas_files

[]

In [16]:
#boucle pour concatener les 3 années

dfall = pd.DataFrame()

for year in gas_files:
    df=pd.read_csv(year, sep=';',header=None, dtype={1: str})

    df.columns = ['id_station','code_postal','type_station','latitude',
                  'longitude','date_releve','type_carburant',
                  'libelle_carburant','prix']
    dfall = pd.concat([dfall, df], axis=0)

#axis = 0 c'est pour concatener dessus dessous    

dfall.shape
assert dfall.shape[1] == 9, "Pas bon nb de colonne"
#nb de ligne = 9 ? pour verifier, sinon met messagage d'erreur

AssertionError: Pas bon nb de colonne

In [ ]:
dfall.shape

In [ ]:
dfstations = pd.read_csv(os.path.join(DATA_DIR,stations_file), sep='|', header=None, encoding='utf-8',dtype={1: str})
dfstations.columns = ['id_station','code_postal','type_station','latitude','longitude','adresse','ville']

In [ ]:
dfstations.head()

In [ ]:
dfservices = pd.read_csv(os.path.join(DATA_DIR,services_file), sep='|', header=None, encoding='utf-8',dtype={1: str})
dfservices.columns = ['id_station','code_postal','type_station','latitude','longitude','services']

In [ ]:
dfservices.head()

In [ ]:
 #sauver nos donnes dans DATA_DIR
    #pickle format efficace
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
dfstations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
dfservices.to_pickle(os.path.join(DATA_DIR, SERVICE_FILE))

In [ ]:
#créer base de données
cnx = sqlite3.connect(GAS_DB)

In [ ]:
dfall.to_sql('prices', cnx, index=False, if exists='replace')

In [ ]:
dfstations.to_sql('stations', cnx, index=False, if exists='replace')
dfservices.to_sql('services', cnx, index=False, if exists='replace')

In [ ]:
pd.read_sql_query('select * from prices where code_postal="13001"', cnx)